## Import

In [20]:
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim.lr_scheduler import _LRScheduler

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from torchvision import models
from torchsummary import summary

# GPU 사용이 가능할 경우, GPU를 사용할 수 있게 함.'
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
print(device)

print(os.environ.get('CUDA_VISIBLE_DEVICES'))

cuda
3


## Utils

In [21]:
# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# 클래스별 IoU를 계산하기 위한 함수
def calculate_iou_per_class(y_true, y_pred, class_id):
    intersection = np.sum((y_true == class_id) & (y_pred == class_id))
    union = np.sum((y_true == class_id) | (y_pred == class_id))
    iou = intersection / union if union > 0 else 0
    return iou

## Dataset, Data Loader

출력이미지 크기 키우기->ex) resnet 2048->1024->512->256 conv 256->512->1024->2048

In [22]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        directory_path = "/mnt/nas27/Dataset/Samsung_DM"
        img_path = self.data.iloc[idx, 1]
        img_path = os.path.join(directory_path, img_path[2:])
        image = cv2.imread(img_path)
        if image is None:
            raise ValueError(f"Failed to load image at {img_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image
        
        mask_path = self.data.iloc[idx, 2]
        mask_path = os.path.join(directory_path, mask_path[2:])
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            raise ValueError(f"Failed to load mask at {mask_path}")
        mask[mask == 255] = 12

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

    
    # 이미지 전처리 클래스
class ImageTransform():
  """
  훈련, 검증 동작 다르게 설정
  이미지 크기 resize, 색상 표준화
  훈련시 RandomResizedCrop, RandomHorizontalFilp으로 데이터 확장
  """
  def __init__(self, resize, mean, std):
    self.data_transform = {
        'train' : transforms.Compose([
            #transforms.RandomResizedCrop(
            #    resize, scale = (0.5, 1.0)), # 데이터 확장
            transforms.RandomHorizontalFlip(), # 데이터 확장
            transforms.ToTensor(), # Tensor로 변환
            transforms.Normalize(mean = mean, std = std) #표준화
        ]),
        'test': transforms.Compose([
            #transforms.Resize(resize), # Resize
            #transforms.CenterCrop(resize), # 중앙을 resize*resize로 crop
            transforms.ToTensor(), # Tensor로 변환
            transforms.Normalize(mean = mean, std = std) # 표준화
        ])

    }
  def __call__(self, img, phase = 'train'):
    """
    phase : 'train' or 'test'
    전처리 모드 지정
    """
    return self.data_transform[phase](img)


transform = A.Compose(
    [   
        #A.Resize(224, 224),
        A.Resize(256, 256),
        A.Normalize(),
        
        # 변형
        A.VerticalFlip(p=0.5),
        # A.RandomRotate90(p=0.5),
        A.HueSaturationValue(p=0.2),
        
        ToTensorV2()
    ]
)

In [ ]:
print(CustomDataset.__len__)

Warmup

In [23]:
class WarmUpLR(_LRScheduler):
    def __init__(self, optimizer, total_iters, last_epoch=-1):
        self.total_iters = total_iters
        super(WarmUpLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        return [base_lr * self.last_epoch / self.total_iters for base_lr in self.base_lrs]


## Define Model

In [24]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1):
        super(Bottleneck, self).__init__()

        # 1x1 convolution
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu1 = nn.ReLU(inplace=True)

        # 3x3 convolution
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu2 = nn.ReLU(inplace=True)

        # 1x1 convolution
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * self.expansion)
            )

        self.relu3 = nn.ReLU(inplace=True)

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu2(out)

        out = self.conv3(out)
        out = self.bn3(out)

        identity = self.shortcut(identity)

        out += identity
        out = self.relu3(out)

        return out

# 예제 사용
block = Bottleneck(64, 64)


In [25]:
#Unet의 기본이 되는 conv블럭
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)  # 여기서 in_channels는 out_channels와 동일해야 합니다.
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu2 = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        return x

#인코더 블럭
class EncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(EncoderBlock,self).__init__() 
        self.convblock1 = ConvBlock(in_channels, out_channels)  # 첫 번째 ConvBlock의 in_channels는 입력 이미지의 채널 수와 일치해야 합니다.
        self.convblock2 = ConvBlock(out_channels, out_channels)  # 두 번째 ConvBlock의 in_channels는 out_channels와 일치해야 합니다.
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self,x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        p = self.maxpool(x)
        return x , p
#디코더 블럭
#디코더는 업샘플링 이후 스킵연결과 붙어서 convblock을 통과해야함
#skip보다 작은 x x먼저 업샘플링 32 -> 64 , skip과 결합 6464 
class DecoderBlock(nn.Module):
    def __init__(self, channels):
        super(DecoderBlock,self).__init__()
        self.upsample = nn.ConvTranspose2d(channels*2, channels, kernel_size=4, stride=2, padding=1)#x 업샘플링
        self.convblock1 = ConvBlock(channels*2, channels)#차원감소
        self.convblock2 = ConvBlock(channels, channels)
    def forward(self,x,skip):
        x = self.upsample(x)
        x = torch.cat([x, skip], dim=1)
        x = self.convblock1(x)
        x = self.convblock2(x)
        return x
        
#Unet구조 middle의 xm값의 움직임에 주의
class Unet(nn.Module):
    def __init__(self,n_classes):
        super(Unet,self).__init__()
        self.encoder1 = EncoderBlock(3,64)
        self.encoder2 = EncoderBlock(64,128)
        self.encoder3 = EncoderBlock(128,256)
        self.encoder4 = EncoderBlock(256,512)
        
        self.middleconv = ConvBlock(512,1024)
        
        
        self.decoder4 = DecoderBlock(512)
        self.decoder3 = DecoderBlock(256)
        self.decoder2 = DecoderBlock(128)
        self.decoder1 = DecoderBlock(64)
        self.segmap = nn.Conv2d(64,n_classes, kernel_size=1)
        
        
    def forward(self,x):
        x1,p = self.encoder1(x)#3->64   #P:256,256 x1 :512,512
        x2,p = self.encoder2(p)#64->128 #P:128,128 x2:256,256
        x3,p = self.encoder3(p)#128->256#p:64,64 x3:128,128
        x4,p = self.encoder4(p)#256->512#p:32,32 x4:64,64
        
        xm = self.middleconv(p)#512->1024#32,32
        
        x = self.decoder4(xm,x4)#뉴런:1024->512->512 #출력tensor:64,64
        x = self.decoder3(x,x3)#뉴런:512->256->256 #출력tensor:128,128
        x = self.decoder2(x,x2)#뉴런:256->128->128 #출력tensor:256,256
        x = self.decoder1(x,x1)#뉴런:128->64->64 #출력tensor:512,512
        x = self.segmap(x)
        x = F.softmax(x,dim=1)
        return x

## ResNet50

In [26]:
import torch.nn as nn
import torchvision.models as models

class UNetWithResnet50Encoder(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        
        # ResNet50 백본을 불러옵니다.
        self.resnet = models.resnet50(pretrained=True)
        
        # 인코더 부분
        self.encoder0 = nn.Sequential(
            self.resnet.conv1,
            self.resnet.bn1,
            self.resnet.relu,
            self.resnet.maxpool
        )
        self.encoder1 = self.resnet.layer1
        self.encoder2 = self.resnet.layer2
        self.encoder3 = self.resnet.layer3
        self.encoder4 = self.resnet.layer4
        
        # 중간 컨볼루션
        self.middle_conv = nn.Sequential(
            nn.Conv2d(2048, 1024, kernel_size=3, padding=1),
            nn.ReLU()
        )
        
        # 디코더 부분
        self.decoder3 = nn.Sequential(
            nn.ConvTranspose2d(1024+1024, 512, kernel_size=2, stride=2),
            nn.ReLU()
        )
        self.decoder2 = nn.Sequential(
            nn.ConvTranspose2d(512+512, 256, kernel_size=2, stride=2),
            nn.ReLU()
        )
        # 디코더 부분
        self.decoder1 = nn.Sequential(
            nn.ConvTranspose2d(256+256, 128, kernel_size=2, stride=2),  # 출력 채널 수를 128로 변경
            nn.ReLU()
        )
        self.decoder0 = nn.Sequential(
            nn.ConvTranspose2d(128+64, n_classes, kernel_size=2, stride=2),  # 입력 채널 수를 192로 변경
            nn.ReLU()
        )
        
    def forward(self, x):
        # 인코더 부분
        x0 = self.encoder0(x)
        x1 = self.encoder1(x0)
        x2 = self.encoder2(x1)
        x3 = self.encoder3(x2)
        x4 = self.encoder4(x3)
        
        # 중간 컨볼루션 후 업샘플링
        middle = self.middle_conv(x4)
        middle_upsampled = nn.functional.interpolate(middle, scale_factor=2, mode='bilinear', align_corners=True)

        # 디코더 부분 (skip connection 추가)
        d3 = self.decoder3(torch.cat([middle_upsampled, x3], dim=1))
        d2 = self.decoder2(torch.cat([d3, x2], dim=1))
        d1 = self.decoder1(torch.cat([d2, x1], dim=1))
        out = self.decoder0(torch.cat([d1, x0], dim=1))
        
        return out


In [27]:
BASE_LR = 0.01
END_LR = 0.01
EP = 30
BATCH_SIZE = 12
ACCMULATION_STEP = 12 
N_CLASSES = 13 #IoU 점수측정하기 위한 클래스의 개수
WUP_ITERS = 10  # 웜업을 위한 반복 횟수
# model 초기화
#model = Unet_resnet18(n_classes = N_CLASSES).to(device)
#model = ResNet50(num_classes=N_CLASSES).to(device)
model = Unet(n_classes = N_CLASSES).to(device)


# loss function과 optimizer 정의
criterion =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=BASE_LR)
optimizer.zero_grad() 
# Warmup을 위한 스케줄러 설정
scheduler_warmup = WarmUpLR(optimizer, WUP_ITERS)

dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./train_source.csv'), transform=transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
valid_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./val_source.csv'), transform=transform)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)

## Model Train

In [28]:
import random
torch.cuda.empty_cache()
for epoch in range(EP):
    # 클래스별 IoU를 누적할 리스트 초기화
    train_class_ious = []
    # 학습
    model.train()
    epoch_loss = 0
    for images, masks in tqdm(dataloader):
        images = images.float().to(device)
        masks = masks.long().to(device)

        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        if (epoch+1) % ACCMULATION_STEP == 0:
            optimizer.step()
            optimizer.zero_grad()
            # Warmup 스케줄러 업데이트
            # if epoch < WUP_ITERS:
            #     scheduler_warmup.step()

        epoch_loss += loss.item()

        # train 클래스별 IoU 계산
        outputs = torch.softmax(outputs, dim=1).cpu()
        outputs = torch.argmax(outputs, dim=1).numpy()

        for class_id in range(N_CLASSES):
            iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
            train_class_ious.append(iou)

    train_class_ious = np.array(train_class_ious).reshape(N_CLASSES, -1)
    train_class_ious = np.mean(train_class_ious, axis=1)
    print("--IoU Scores Train--")
    for class_id, iou in enumerate(train_class_ious):
        print(f'Class{class_id}: {iou:.4f}', end=" ")
        if (class_id+1) % 6 == 0:
            print()

    # mIoU 계산
    train_mIoU = np.mean(train_class_ious)

    # validation
    val_loss = 0
    val_class_ious = []  # 클래스별 IoU를 누적할 리스트 초기화
    with torch.no_grad():
        model.eval()
        for images, masks in tqdm(valid_dataloader):
            images = images.float().to(device)
            masks = masks.long().to(device)
            outputs = model(images)

            # validation loss 계산
            val_loss += criterion(outputs, masks).item()

            # validation 클래스별 IoU 계산
            outputs = torch.softmax(outputs, dim=1).cpu()
            outputs = torch.argmax(outputs, dim=1).numpy()

            for class_id in range(N_CLASSES):
                iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
                val_class_ious.append(iou)

    val_class_ious = np.array(val_class_ious).reshape(N_CLASSES, -1)
    val_class_ious = np.mean(val_class_ious, axis=1)
    print("--IoU Scores Valid--")
    for class_id, iou in enumerate(val_class_ious):
        print(f'Class{class_id}: {iou:.4f}', end=" ")
        if (class_id+1) % 6 == 0:
            print("")

    # mIoU 계산
    val_mIoU = np.mean(val_class_ious)

    # 에폭마다 결과 출력 
    print(f"\nEpoch{epoch+1}")
    print(f"Train Loss: {(epoch_loss/len(dataloader))}, Train mIoU Score: {train_mIoU:.4f}")
    print(f"Validation Loss: {val_loss/len(valid_dataloader)}, Validation mIoU Score: {val_mIoU:.4f}")
    print("___________________________________________________________________________________________\n")


100%|██████████| 183/183 [02:53<00:00,  1.05it/s]


--IoU Scores Train--
Class0: 0.0143 Class1: 0.0137 Class2: 0.0141 Class3: 0.0141 Class4: 0.0138 Class5: 0.0137 
Class6: 0.0142 Class7: 0.0134 Class8: 0.0141 Class9: 0.0138 Class10: 0.0138 Class11: 0.0145 
Class12: 0.0139 

100%|██████████| 39/39 [00:20<00:00,  1.86it/s]


--IoU Scores Valid--
Class0: 0.0145 Class1: 0.0136 Class2: 0.0131 Class3: 0.0139 Class4: 0.0136 Class5: 0.0142 
Class6: 0.0119 Class7: 0.0139 Class8: 0.0144 Class9: 0.0128 Class10: 0.0133 Class11: 0.0129 
Class12: 0.0136 
Epoch1
Train Loss: 2.5714878045796046, Train mIoU Score: 0.0139
Validation Loss: 2.5718402984814768, Validation mIoU Score: 0.0135
___________________________________________________________________________________________



100%|██████████| 183/183 [02:54<00:00,  1.05it/s]


...

--IoU Scores Train--
Class0: 0.0140 Class1: 0.0139 Class2: 0.0144 Class3: 0.0139 Class4: 0.0138 Class5: 0.0140 
Class6: 0.0141 Class7: 0.0139 Class8: 0.0140 Class9: 0.0135 Class10: 0.0142 Class11: 0.0144 
Class12: 0.0141 

100%|██████████| 39/39 [00:32<00:00,  1.21it/s]


--IoU Scores Valid--
Class0: 0.0145 Class1: 0.0135 Class2: 0.0135 Class3: 0.0132 Class4: 0.0130 Class5: 0.0136 
Class6: 0.0128 Class7: 0.0134 Class8: 0.0149 Class9: 0.0127 Class10: 0.0124 Class11: 0.0139 
Class12: 0.0136 
Epoch6
Train Loss: 2.571506811621411, Train mIoU Score: 0.0140
Validation Loss: 2.5719841015644564, Validation mIoU Score: 0.0135
___________________________________________________________________________________________



 23%|██▎       | 42/183 [00:47<02:40,  1.14s/it]


KeyboardInterrupt: 

## Inference

In [ ]:
# test_dataset = CustomDataset(csv_file='./test.csv', transform=transform, infer=True)
# test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

In [ ]:
# with torch.no_grad():
#     model.eval()
#     result = []
#     for images in tqdm(test_dataloader):
#         images = images.float().to(device)
#         outputs = model(images)
#         outputs = torch.softmax(outputs, dim=1).cpu()
#         outputs = torch.argmax(outputs, dim=1).numpy()
#         # batch에 존재하는 각 이미지에 대해서 반복
#         for pred in outputs:
#             pred = pred.astype(np.uint8)
#             pred = Image.fromarray(pred) # 이미지로 변환
#             pred = pred.resize((960, 540), Image.NEAREST) # 960 x 540 사이즈로 변환
#             pred = np.array(pred) # 다시 수치로 변환
#             # class 0 ~ 11에 해당하는 경우에 마스크 형성 / 12(배경)는 제외하고 진행
#             for class_id in range(12):
#                 class_mask = (pred == class_id).astype(np.uint8)
#                 if np.sum(class_mask) > 0: # 마스크가 존재하는 경우 encode
#                     mask_rle = rle_encode(class_mask)
#                     result.append(mask_rle)
#                 else: # 마스크가 존재하지 않는 경우 -1
#                     result.append(-1)
        

## Submission

In [ ]:
# submit = pd.read_csv('./sample_submission.csv')
# submit['mask_rle'] = result
# submit

In [ ]:
# submit.to_csv('./baseline_submit.csv', index=False)